# ADS 599: Retail Dataset Capstone Project
#### By Jesse Gutierrez, Verity Pierson, & Sultan Mahmud Rahat

##### Load Req. Packages

In [ ]:
import pandas as pd
import sqlite3

#### While the dataset is complete containing all necessary attributes, we want to mimic data sources as expected to be found. Therefore, we split the data into subsets and assigned primary/foreign keys based on what would be anticipated.

In [3]:
# Load csv file downloaded from kaggle
df = pd.read_csv("Datasets/retail_data.csv")

# Display output
df.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [5]:
# Subset data based on data sources (receiving, sales, website, etc.)
customer_info = df[['Customer_ID', 'Name', 'Email', 'Phone', 'Address', 'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income', 'Customer_Segment']]
transaction_details = df[['Transaction_ID', 'Customer_ID', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases', 'Amount', 'Total_Amount']]
transaction_logistics = df[['Transaction_ID', 'Shipping_Method', 'Payment_Method', 'Order_Status']]

# Generate a Product_ID for each unique product w/ leading zeros
unique_products = df['products'].unique()
product_mapping = {product: f'P{idx:03}' for idx, product in enumerate(unique_products)}

# Map the Product_IDs back to the dataframe
df['Product_ID'] = df['products'].map(product_mapping)

# Create product_info w/ Product_ID & w/o Transaction_ID
product_info = df[['Product_ID', 'Product_Category', 'Product_Brand', 'Product_Type', 'products']]

# Add Product_ID to feedback
feedback = df[['Product_ID', 'Feedback', 'Ratings']]

# Verify the data
customer_info.head()

,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,Age,Gender,Income,Customer_Segment
0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,21.0,Male,Low,Regular
1,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,19.0,Female,Low,Premium
2,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,48.0,Male,Low,Regular
3,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,56.0,Male,High,Premium
4,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,22.0,Male,Low,Premium


In [6]:
# Display the range of Product_IDs
product_ids = df['Product_ID'].unique()
print("Range of Product_IDs:", min(product_ids), "to", max(product_ids))

# Count of unique Product_IDs
print("Total unique Product_IDs:", len(product_ids))

# Verify the number of unique products
print("Total unique products:", len(unique_products))

Range of Product_IDs: P000 to P317
Total unique Product_IDs: 318
Total unique products: 318


#### Once the data has been split, we are going to use a local database using SQLite

In [2]:
# Pathway to the datasets
datasets_path = '/Users/jesse/Desktop/School/University of San Diego/ADS 599- Capstone/Retail_Analysis/Datasets/'

# CSV files
csv_files = {
    'customer_info': 'customer_info.csv',
    'transaction_details': 'transaction_details.csv',
    'product_info': 'product_info.csv',
    'feedback': 'feedback.csv',
    'transaction_logistics': 'transaction_logistics.csv'
}

# Create a connection to the SQLite database
conn = sqlite3.connect('retail_analysis.db')
cursor = conn.cursor()

# Load each CSV into the SQLite database
for table_name, csv_file in csv_files.items():
    df = pd.read_csv(datasets_path + csv_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Loaded {csv_file} into table {table_name}")

# Fx to get column names & types
def get_table_info(table_name):
    query = f'PRAGMA table_info({table_name});'
    result = pd.read_sql_query(query, conn)
    return result[['name', 'type']]

# Get the list of table names
table_names_query = "SELECT name FROM sqlite_master WHERE type='table';"
table_names = pd.read_sql_query(table_names_query, conn)

# Print table names w/ column names & types
for table_name in table_names['name']:
    print(f"Table: {table_name}")
    table_info = get_table_info(table_name)
    print(table_info)
    print("\n")

# Close the connection
conn.close()

Loaded customer_info.csv into table customer_info
Loaded transaction_details.csv into table transaction_details
Loaded product_info.csv into table product_info
Loaded feedback.csv into table feedback
Loaded transaction_logistics.csv into table transaction_logistics
Table: customer_info
                name  type
0        Customer_ID  REAL
1               Name  TEXT
2              Email  TEXT
3              Phone  REAL
4            Address  TEXT
5               City  TEXT
6              State  TEXT
7            Zipcode  REAL
8            Country  TEXT
9                Age  REAL
10            Gender  TEXT
11            Income  TEXT
12  Customer_Segment  TEXT


Table: transaction_details
              name  type
0   Transaction_ID  REAL
1      Customer_ID  REAL
2             Date  TEXT
3             Year  REAL
4            Month  TEXT
5             Time  TEXT
6  Total_Purchases  REAL
7           Amount  REAL
8     Total_Amount  REAL


Table: product_info
               name  type
0       

#### Exploratory Data Analysis